# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [24]:
import requests
from requests.structures import CaseInsensitiveDict

url = "https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316&limit=20&apiKey=434379c08c4c4db394f98b8fd2fabd03"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_cities_df = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",  # Size of the points based on humidity
    scale=0.8,
    color="Humidity",  # Color of the points based on humidity
    cmap='viridis'     # Colormap for better visualization
).opts(
    scalebar=True,
    tools=['hover'],  # Enable hover tool for tooltips
    title="City Humidity Levels",  # Title for the plot
    width=1000,  # Width of the plot
    height=600  # Height of the plot
)

# Display the map
map_plot_cities_df


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:

# Narrow down cities that fit criteria and drop any results with null values
cities_criteria = city_data_df.loc[city_data_df['Cloudiness'] == 0,:]

# Drop any rows with null values
clean_cities = cities_criteria.dropna()

# Display sample data
clean_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
32,32,cape town,-33.9258,18.4232,20.61,71,0,9.26,ZA,1666108250
44,44,collierville,35.0420,-89.6645,7.11,47,0,5.66,US,1666108151
...,...,...,...,...,...,...,...,...,...,...
542,542,bilma,18.6853,12.9164,31.90,18,0,4.17,NE,1666108548
546,546,arcata,40.8665,-124.0828,10.66,93,0,1.54,US,1666108260
550,550,pali,25.7667,73.3333,27.18,26,0,1.38,IN,1666108552
559,559,lompoc,34.6391,-120.4579,17.55,100,0,3.60,US,1666108556


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_cities[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
1,farsund,NO,58.0948,6.8047,100,
7,carnarvon,AU,-24.8667,113.6333,76,
9,saint-pierre,RE,-21.3393,55.4781,68,
32,cape town,ZA,-33.9258,18.4232,71,
44,collierville,US,35.0420,-89.6645,47,
...,...,...,...,...,...,...
542,bilma,NE,18.6853,12.9164,18,
546,arcata,US,40.8665,-124.0828,93,
550,pali,IN,25.7667,73.3333,26,
559,lompoc,US,34.6391,-120.4579,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "categories":"accommodation.hotel",
    "limit":limit,
    "apiKey":geoapify_key
}
# print(params["apikey"])

# for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
index = 1
Latitude = hotel_df.loc[index, "Lat"]
longitude = hotel_df.loc[index, "Lng"]

print(longitude, Latitude)

# Add the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{longitude},{Latitude},{radius}"
params["bias"] = f"proximity:{longitude},{Latitude}"
#https://api.geoapify.com/v2/places?
# categories=commercial.supermarket
# &filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316
# &limit=20
# &apiKey=434379c08c4c4db394f98b8fd2fabd03
    # Assuming you have a 'City' column
    # Set base URL
base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
name_address = response.json()
name_address

6.8047 58.0948


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Rederiet Hotell',
    'country': 'Norway',
    'country_code': 'no',
    'county': 'Agder',
    'city': 'Farsund',
    'municipality': 'Farsund',
    'postcode': '4550',
    'district': 'Vestersiden',
    'street': 'Torvgaten',
    'lon': 6.8048004,
    'lat': 58.0955625,
    'formatted': 'Rederiet Hotell, Torvgaten, 4550 Farsund, Norway',
    'address_line1': 'Rederiet Hotell',
    'address_line2': 'Torvgaten, 4550 Farsund, Norway',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database License',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Rederiet Hotell',
      'osm_id': 1363069719,
      'amenity': 'cafe;restaurant',
      'tourism': 'hotel',
      'osm_type': 'n'}},
    'distance': 85,
    'place_id': '51

In [30]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
# filters = f"circle:{lng},{lat},{radius}"
# bias = f"proximity:{lng},{lat}"
params = {
    "categories":"accommodation.hotel",
    "limit":limit,
    "apiKey":geoapify_key
}
# Set base URL
# base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{Latitude}"
    # Assuming you have a 'City' column
    # City = hotel_df.loc[index, "City"]
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
farsund - nearest hotel: Rederiet Hotell
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
cape town - nearest hotel: Townhouse Hotel
collierville - nearest hotel: Days Inn & Suites by Wyndham Collierville Germantown Area
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: No hotel found
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
qostanay - nearest hotel: Мини гостиница
esperance - nearest hotel: Pier Hotel
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
cedar city - nearest hotel: Baymont Inn & Suites
lichuan - nearest hotel: No hotel found
half moon bay - nearest hotel: Half Moon Bay Inn
brenham - nearest hotel: Ant Street Inn
puerto del rosario - nearest hotel: Hotel Tamasite
kahului - nearest hotel: Maui Seaside Hotel
geraldton - nearest hotel: O

,City,Country,Lat,Lng,Humidity,Hotel Name
1,farsund,NO,58.0948,6.8047,100,Rederiet Hotell
7,carnarvon,AU,-24.8667,113.6333,76,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
32,cape town,ZA,-33.9258,18.4232,71,Townhouse Hotel
44,collierville,US,35.0420,-89.6645,47,Days Inn & Suites by Wyndham Collierville Germ...
...,...,...,...,...,...,...
542,bilma,NE,18.6853,12.9164,18,No hotel found
546,arcata,US,40.8665,-124.0828,93,Hotel Arcata
550,pali,IN,25.7667,73.3333,26,Hotel Pawan International
559,lompoc,US,34.6391,-120.4579,100,Red Roof Inn Lompoc


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",  # Longitude
    "Lat",  # Latitude
    geo=True,
    tiles="OSM",  # You can choose other tile options
    frame_width=800,
    frame_height=600,
    color='Hotel Name',  # Optionally color by hotel name
    size=40  # You can adjust size based on other metrics if needed
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)